# Authentication Simulation (Production Model)

This notebook demonstrates the real-time inference pipeline of the gait authentication system in production folder.

Objective of file :- A clean demonstration of production inference logic which is covered in production folder.

Configuration:

- Model: Random Forest (800 trees)
- Dataset: RealWorldLive (5 users)
- Window size: 128 samples
- Step size: 32 samples (75% overlap)
- Sampling rate: ~20 Hz
- Window duration: 6.4 seconds
- Voting threshold: 0.45
- Features per window: 56

This notebook simulates what the production server performs when a CSV file is received from the mobile application.


In [1]:
import numpy as np
import pandas as pd
import joblib
from scipy.fft import fft
from collections import Counter

WINDOW_SIZE = 128
STEP_SIZE = 32
THRESHOLD = 0.45

MODEL_PATH = "../../production/model/rf_model.pkl"
SCALER_PATH = "../../production/model/scaler.pkl"
CSV_PATH = "../../production/received_gait.csv"



In [2]:
model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

print("Model and scaler loaded successfully.")


Model and scaler loaded successfully.


In [3]:
df = pd.read_csv(CSV_PATH)

expected_cols = ["ax", "ay", "az", "wx", "wy", "wz"]

if not all(col in df.columns for col in expected_cols):
    raise ValueError("ACCESS_DENIED (bad csv format)")

df = df[expected_cols].apply(pd.to_numeric, errors="coerce").dropna()

print("CSV validated.")
print("Total samples:", len(df))


CSV validated.
Total samples: 571


In [4]:
motion_energy = df[["ax", "ay", "az"]].std().mean()

print("Motion energy:", motion_energy)

is_static = motion_energy < 0.15

if is_static:
    print("ACCESS_DENIED (static)")


Motion energy: 0.006682463641269315
ACCESS_DENIED (static)


In [6]:
def sliding_windows(data, window_size=128, step=32):
    windows = []
    for start in range(0, len(data) - window_size + 1, step):
        windows.append(data[start:start+window_size])
    return windows

windows = sliding_windows(df.values, WINDOW_SIZE, STEP_SIZE)

print("Windows generated:", len(windows))


Windows generated: 14


In [7]:
def gait_features(signal):
    mean = np.mean(signal)
    std = np.std(signal)
    rms = np.sqrt(np.mean(signal**2))
    ptp = np.ptp(signal)

    fft_vals = np.abs(fft(signal))
    fft_vals = fft_vals[:len(fft_vals)//2]

    dom_freq = np.argmax(fft_vals)
    spec_energy = np.sum(fft_vals**2)
    freq_std = np.std(fft_vals)

    return [mean, std, rms, ptp, dom_freq, spec_energy, freq_std]

X = []

for window in windows:
    features = []

    acc = window[:, 0:3]
    gyro = window[:, 3:6]

    acc_mag = np.linalg.norm(acc, axis=1)
    gyro_mag = np.linalg.norm(gyro, axis=1)

    for axis in range(3):
        features += gait_features(acc[:, axis])
    features += gait_features(acc_mag)

    for axis in range(3):
        features += gait_features(gyro[:, axis])
    features += gait_features(gyro_mag)

    X.append(features)

X = np.array(X)

print("Feature matrix shape:", X.shape)


Feature matrix shape: (14, 56)


In [8]:
if is_static:
    print("\nAuthentication stopped due to static detection.")
elif len(X) == 0:
    print("ACCESS_DENIED (insufficient data)")
else:
    X_scaled = scaler.transform(X)
    preds = model.predict(X_scaled)

    vote_counts = Counter(preds)
    majority_person, votes = vote_counts.most_common(1)[0]
    vote_ratio = votes / len(preds)

    print("Window Predictions:", vote_counts)
    print("Majority Person:", majority_person)
    print("Vote Ratio:", round(vote_ratio, 3))

    if vote_ratio >= THRESHOLD:
        print(f"\nACCESS_GRANTED (Person{majority_person})")
    else:
        print(f"\nACCESS_DENIED (closest: Person{majority_person})")



Authentication stopped due to static detection.
